https://jira.x5.ru/browse/CVMXC-2630

In [1]:
name = 'CVMXC-2630_mareven'

In [2]:
import sys
import pandas as pd
import pyspark.sql.functions as F
sys.path.append('/home/jovyan/glow-byte-filters-pyspark')
sys.path.append('/home/jovyan/x5_some_tasks/Osipov/')
import datetime
from datetime import timedelta
from spark import *
from utils_osipov import *
from logic_filters import *
from segmentation import *

%load_ext autoreload
%autoreload 2

pd.options.display.float_format = '{:,.2f}'.format
pd.options.display.max_rows = 500

In [3]:
sc, spark = restart_spark(name, 100, executor_memory="3G", executor_cores=5, driver_memory="12G",
                          additional_params={"spark.sql.shuffle.partitions": "200"})
sc.setLogLevel('ERROR')

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


In [4]:
LOYALTY_CARDS = "hive_ssa_tc5.loyalty_card"
LOYALTY_CARDHOLDERS = "hive_ssa_tc5.loyalty_cardholder"
ACCOUNTS = "hive_ssa_tc5.account"
CVM5_GUESTS = "hive_cvm_acrm.cvm5_guests"

DIM_STORE = "hive_ssa_main.dim_store"
CHECKS_HEADERS = "hive_ssa_main.fct_rtl_txn"
CHECKS_ITEMS = "hive_ssa_main.fct_rtl_txn_item"
PRODUCTS = "hive_ssa_tc5.cvm_product"

### Выбираем гостей нужного юзкейса

In [5]:
usecase = ['cross', 'upgrade', 'ump']
lifetime = 90
freq = 1
dt = datetime.date(2022, 2, 3)
base_qty_per_period = 3

In [6]:
customers_usecase = (spark
                     .table(CVM5_GUESTS)
                     .filter(F.to_date('calculation_dt') == dt)
                     .filter(F.col('usecase').isin(usecase))
                     .filter(F.col('lifetime') >= lifetime)
                     .filter(F.col('frequency') >= freq)
                     .select('account_no', 'customer_rk')
                            )

In [ ]:
customers_usecase.count() # 15 857 096

### Проверяем на доступность отобранных гостей в определенную дату

In [7]:
check_date = '2022-02-24'

In [ ]:
seg_sms1 = (sms_channel_filters_glowbyte(spark=spark,
                                         guests=customers_usecase, 
                                         usecase_name=usecase[0], 
                                         check_date=check_date, 
                                         debug_mode=False)
                                    .select('customer_rk')
                                    .distinct()
           )

In [ ]:
seg_sms2 = (sms_channel_filters_glowbyte(spark=spark,
                                         guests=customers_usecase, 
                                         usecase_name=usecase[1], 
                                         check_date=check_date, 
                                         debug_mode=False)
                                    .select('customer_rk')
                                    .distinct()
           )

In [ ]:
seg_sms3 = (sms_channel_filters_glowbyte(spark=spark,
                                         guests=customers_usecase, 
                                         usecase_name=usecase[2], 
                                         check_date=check_date, 
                                         debug_mode=False)
                                    .select('customer_rk')
                                    .distinct()
           )

In [ ]:
seg_sms = seg_sms1.union(seg_sms2).union(seg_sms3).distinct()

In [8]:
# seg_sms.write.parquet('temp2630_1', mode='overwrite')
seg_sms = spark.read.parquet('temp2630_1')

In [9]:
seg_sms.count() # 5 814 919

5814919

### Проверка на доступность плю в магазинах

РФ

In [10]:
plu_codes = [76002, 3446162]

In [11]:
pplu = spark.createDataFrame(pd.DataFrame({"plu_code": plu_codes}))

shops = (spark.table(DIM_STORE)
            .filter(F.col('valid_to_dttm')== datetime.datetime(5999, 1, 1, 0, 0))
            #.filter(F.col('federal_subject_dk').isin([23, 93, 123, 193, 74, 174,
             #                                         774, 16, 116, 716, 71, 32]))
#             .filter(F.col('macroregion_dk').isin(macroregion_dk))
            .selectExpr('store_id as plant')
            .distinct()
            #.toPandas()['plant']
            #.tolist()
            )


tc5_stores_assort = (
    spark.table('HIVE_SSA_MAIN.ASSORTMENT_X_PLU_X_STORE')
    .withColumnRenamed('plu_id', 'plu_code')
    .withColumnRenamed('store_id', 'plant')
    .filter(F.col("plu_negate_flg") != 1)
    .selectExpr('plu_code', 'plant')
    .join(F.broadcast(pplu), 'plu_code', 'inner')
)

plants = (shops.join(tc5_stores_assort, on='plant', how='inner')
                .select('plant')
                .distinct()
                .toPandas()['plant']
                .tolist()
         )

len(plants)

21676

### Собираем чеки этих гостей и оставляем тех, кто покупал продукты с synthetic_catalog_id

In [12]:
start_date = datetime.date(2021, 11, 3)
end_date = datetime.date(2022, 2, 2)

In [13]:
checks_headers = (spark.table(CHECKS_HEADERS)
                      .filter(F.col('rtl_txn_dt').between(start_date, end_date))
                      .filter((F.col('loyalty_card_no') != '') & (F.col('loyalty_card_no').isNotNull()))
                      .filter(F.col('financial_unit_format_dk') == 'D')
                      .filter(F.col('rtl_txn_cancel_flg') == 0)
                      .select('rtl_txn_id', 'loyalty_card_no', 'store_id')
                     )


In [14]:
checks_headers_tc5 = checks_headers.filter(F.col('store_id').isin(plants)) #оставили чеки только с нужными магазинами

In [15]:
loyalty_cards = (spark
                    .table(LOYALTY_CARDS)
                    .withColumnRenamed("loyalty_card_id", "loyalty_card_no")
                    .withColumnRenamed("loyalty_account_id", "account_no")
                    .withColumnRenamed("loyalty_account_acrm_id", "account_rk")
                    .select('account_no', 'loyalty_card_no')
                )
loyalty_cardholders = (spark
                        .table(LOYALTY_CARDHOLDERS)
                        .withColumnRenamed("loyalty_cardholder_acrm_id", "customer_rk")
                        .withColumnRenamed("loyalty_account_id", "account_no")
                        .select('account_no', 'customer_rk')
                      )
clients_info = loyalty_cards.join(loyalty_cardholders, on='account_no', how='inner')
clients_info = clients_info.join(seg_sms, on='customer_rk', how='inner')

In [16]:
checks_headers_tc5 = checks_headers_tc5.join(clients_info, on='loyalty_card_no') #оставили чеки только нужных гостей

In [17]:
checks_items = (spark.table(CHECKS_ITEMS) 
                    .withColumnRenamed('plu_id', 'plu_code')
                    .withColumnRenamed('turnover_no_vat_amt', 'zsalnovat')
                    .withColumnRenamed('turnover_vat_rub_amt', 'zsale_vat')
                    .withColumnRenamed('prime_cost_no_vat_amt', 'zcst_n')
                    .withColumnRenamed('turnover_base_uom_amt', 'base_qty')
                    .withColumnRenamed('discount_amt', 'zdiscount')
                    .withColumnRenamed('fact_regular_promo_flg', 'zpromofl')
                    .filter(F.col('rtl_txn_dt').between(start_date, end_date))
                    .filter((F.col('zsalnovat') >= 0) & (F.col('base_qty') >= 0) & (F.col('zcst_n') > 0)) #keep only correct data
                        .select('plu_code' #items id
                                , 'rtl_txn_id' #cheques id
                               , 'base_qty'
                               )
                   )


In [18]:
# plu_hierarchy_lvl_4_dk = ['FR0604001', 'FR0608005', 'FR0604002', 'FR0608001'] #сметана, молоко, кефир
syntethic_category_id = [96,57,75,73,101,55,56,22]
# plu_brand_code = ['5727','M082','M083','5728']
# plu_vendor_nm = ['Данон']
# plu_not_in = [4138521]

In [19]:
plu_codes_cat = (spark
                 .table(PRODUCTS)
#                  .filter(F.col('plu_hierarchy_lvl_4_dk').isin(plu_hierarchy_lvl_4_dk))
#                  .filter(F.col('plu_vendor_nm').isin(plu_vendor_nm))
                 .filter(F.col('syntethic_category_id').isin(syntethic_category_id))
#                  .filter(~F.col('plu_id').isin(plu_not_in))
                 .select('plu_id')
                 .distinct()
                 .toPandas()['plu_id']
                 .tolist()
                )

In [20]:
checks_items = checks_items.filter(F.col('plu_code').isin(plu_codes_cat)) # только чеки с нужными plu

In [21]:
checks_tc5 = checks_items.join(checks_headers_tc5, 'rtl_txn_id', how='inner')

In [22]:
accs = ['account_no']
pdf = checks_tc5.groupby(accs).agg(F.sum('base_qty').alias('base_qty_per_period'))
pdf = pdf.filter(F.col('base_qty_per_period') >= base_qty_per_period)
checks_tc5 = checks_tc5.join(pdf, on=accs)

In [23]:
# те, кто покупал во 2 период даниссимо
seg = (checks_tc5.select('customer_rk')
                  .distinct()
      )

In [24]:
seg.write.parquet('temp2630_seg1', mode='overwrite')
seg = spark.read.parquet('temp2630_seg1')

In [25]:
seg.count()

5324031

In [26]:
seg_pd1 = seg.toPandas()

In [27]:
len(seg_pd1)

5324031

In [28]:
seg_no = 'seg1'

In [29]:
seg_pd1.to_csv(name + '_' + seg_no + '_All_uc' + str(dt) + '.csv', index=False)

In [30]:
seg_pd1['customer_rk'] = seg_pd1['customer_rk'].astype('str')

In [33]:
cross, upgrade, ump = get_usecases(seg_pd1)

In [34]:
print('cross: {}, upgrade: {}, ump: {} (all: {})'.format(len(cross), len(upgrade), len(ump), len(cross)+len(upgrade)+len(ump)))

cross: 1804835, upgrade: 1851904, ump: 1667292 (all: 5324031)
